In [1]:
# pip install --timeout=1000 sentence-transformers

In [2]:
#Install Packages
# !pip install faiss-cpu

In [3]:
import pandas as pd

pd.set_option('display.max_colwidth', 100)




In [4]:
df = pd.read_csv("sample_text.csv")
df.shape

(8, 2)

In [5]:
df

text  \
0                                        Meditation and yoga can improve mental health   
1                     Fruits, whole grains and vegetables helps control blood pressure   
2                                    These are the latest fashion trends for this week   
3                                    Vibrant color jeans for male are becoming a trend   
4                                                   The concert starts at 7 PM tonight   
5                      Navaratri dandiya program at Expo center in Mumbai this october   
6                                    Exciting vacation destinations for your next trip   
7  Maldives and Srilanka are gaining popularity in terms of low budget vacation places   

  category  
0   Health  
1   Health  
2  Fashion  
3  Fashion  
4    Event  
5    Event  
6   Travel  
7   Travel

Creating source embeddings for the text column

In [8]:
from sentence_transformers import SentenceTransformer

In [9]:
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text)

c:\Users\ansar\anaconda3\envs\research\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ansar\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [10]:
vectors.shape

(8, 768)

In [11]:
vectors

array([[-0.00247395,  0.03626721, -0.05290459, ..., -0.09152356,
        -0.03970002, -0.04330488],
       [-0.03357267,  0.00980517, -0.03250129, ..., -0.05165467,
         0.02245886, -0.03156182],
       [-0.01865323, -0.04051314, -0.01235387, ...,  0.00610585,
        -0.07179647,  0.02773853],
       ...,
       [-0.00066458,  0.04252125, -0.05645508, ...,  0.01315471,
        -0.03183568, -0.04357664],
       [-0.03317154,  0.03252455, -0.02484838, ...,  0.01174421,
         0.05747123,  0.00571021],
       [-0.00166393,  0.00413829, -0.04597083, ...,  0.02008528,
         0.05656242, -0.00161596]], dtype=float32)

In [12]:
dim = vectors.shape[1]
dim

768

Build a FAISS Index for vectors

In [13]:
import faiss

index = faiss.IndexFlatL2(dim)

Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [14]:
index.add(vectors)

Encode search text using same encorder and normalize the output vector

In [17]:
# search_query = "I want to buy a simple t-shirt"
search_query = "looking for places to visit during the holidays"
# search_query = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query)
vec.shape

(768,)

In [18]:
import numpy as np
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

Search for similar vector in the FAISS index created

In [20]:
index.search(svec,k=3)

(array([[0.9273728, 1.1601745, 1.7490072]], dtype=float32),
 array([[6, 7, 2]], dtype=int64))

In [23]:
distances, I = index.search(svec, k=2)
distances

array([[0.9273728, 1.1601745]], dtype=float32)

In [24]:
I,I.tolist()

(array([[6, 7]], dtype=int64), [[6, 7]])

In [25]:
row_indices = I.tolist()[0]
row_indices

[6, 7]

In [26]:
df.loc[row_indices]

text  \
6                                    Exciting vacation destinations for your next trip   
7  Maldives and Srilanka are gaining popularity in terms of low budget vacation places   

  category  
6   Travel  
7   Travel